In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# read data and add color column

In [2]:
cd 'drive/My Drive/visual'

/content/drive/My Drive/visual


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('data.csv')
provinceArray = ["新疆","西藏","内蒙古","青海","四川","黑龙江","甘肃","云南","广西","湖南","陕西","广东","吉林","河北","湖北","贵州","山东","江西","河南","辽宁","山西","安徽","福建","浙江","江苏","重庆","宁夏","海南","台湾","北京","天津","上海","香港","澳门"]
ind = dict(zip(list(range(34)), provinceArray))
data = data.rename(index = ind)

# draw a map

In [5]:
cd 'bokeh'

/content/drive/My Drive/visual/bokeh


In [0]:
class China:
	provinceArray = ["新疆","西藏","内蒙古","青海","四川","黑龙江","甘肃","云南","广西","湖南","陕西","广东","吉林","河北","湖北","贵州","山东","江西","河南","辽宁","山西","安徽","福建","浙江","江苏","重庆","宁夏","海南","台湾","北京","天津","上海","香港","澳门"]
	provinceArray2Hebei = ["新疆","西藏","内蒙古","青海","四川","黑龙江","甘肃","云南","广西","湖南","陕西","广东","吉林","河北","河北","湖北","贵州","山东","江西","河南","辽宁","山西","安徽","福建","浙江","江苏","重庆","宁夏","海南","台湾","北京","天津","上海","香港","澳门"]

In [0]:
import json
import numpy as np
# from china import China

class ChinaMap:
	provinceArray = China().provinceArray

	def __init__(self):
		f = open("chinamap.json", 'r')
		geojson = f.read()
		self.geoinfo = json.loads(geojson)

	def allProvinces(self):
		xs = []
		ys = []
		for i in range(len(self.provinceArray)):
			if self.provinceArray[i] == "河北":
				x, y = self.hebei()
				xs.extend(x)
				ys.extend(y)
			else:
				x, y = self.province(self.provinceArray[i])
				xs.append(x)
				ys.append(y)
		return xs,ys

	def selectedProvinces(self,selectedArray):
		xs = []
		ys = []
		for i in range(len(selectedArray)):
			if selectedArray[i] == "河北":
				x, y = self.hebei()
				xs.extend(x)
				ys.extend(y)
			else:
				x, y = self.province(selectedArray[i])
				xs.append(x)
				ys.append(y)
		return xs,ys

	def province(self, name):
		x = []
		y = []
		if name in self.provinceArray:
			index = self.provinceArray.index(name)
			coordinates = self.geoinfo["features"][index]["geometry"]["coordinates"][0]
			for i in range(len(coordinates)):
				x.append(coordinates[i][0])
				y.append(coordinates[i][1])

		return x, y

	def hebei(self):
		xs = []
		ys = []
		index = self.provinceArray.index("河北")
		for j in range(2):
			x = []
			y = []
			coordinates = self.geoinfo["features"][index]["geometry"]["coordinates"][j][0]
			for i in range(len(coordinates)):
				x.append(coordinates[i][0])
				y.append(coordinates[i][1])
			xs.append(x)
			ys.append(y)
		
		return xs, ys

In [8]:
from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Range1d, HoverTool, CustomJS
from bokeh.layouts import widgetbox,column,row, gridplot, layout
from bokeh.models.widgets import Dropdown, TextInput, Button, RadioButtonGroup, Select, Slider, DataTable, DateFormatter, TableColumn
import json
import numpy as np
# from china import China
# from chinamap import ChinaMap
from datetime import date
from random import randint


time = '2020-02-10'

china = China()
provinceArray = china.provinceArray
provinceArray2Hebei = china.provinceArray2Hebei
chinamap = ChinaMap()
xs, ys = chinamap.allProvinces()


def get_color_legends(row, column):
    if data.loc[row,column]<100:
        c = 'bisque'
        l = '1-99'
    elif data.loc[row,column]<500:
        c = 'lightsalmon'
        l = '100-499'
    elif data.loc[row,column]<1000:
        c = 'tomato'
        l = '500-999'
    elif data.loc[row,column]<10000:
        c = 'crimson'
        l = '1000-9999'
    else:
        c = 'red'
        l = 'more than 10000'
    return c,l

legends,colors = [], []
col = time + '_Confirmed'
for x in range(35):
    row = provinceArray2Hebei[x]
    c,l = get_color_legends(row, col)
    legends.append(l)
    colors.append(c)

source = ColumnDataSource(
        data = dict(
            x = xs,
            y = ys,
            provinces = china.provinceArray2Hebei,
            legends = legends,
            colors = colors
        )
    )

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@provinces</span>
            </div>
        </div>
        """
    )

p = figure(toolbar_location="below",
           toolbar_sticky=False,tools = [hover, "pan","wheel_zoom","box_zoom","reset","save"])
p.toolbar.logo = None
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.y_range = Range1d(10, 65)
p.x_range = Range1d(70, 140)
p.patches('x', 'y',legend='legends',color = "colors", source=source, alpha=0.5, line_width=2,line_color = "white")

'''
text_input = TextInput(value="default", title="标题:")
slider = Slider(start=0, end=10, value=1, step=.1, title="Slider")
button_group = RadioButtonGroup(labels=["Option 1", "Option 2", "Option 3"], active=0)
select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
button_1 = Button(label="Button 1")
'''

callback = CustomJS(args=dict(source=source), code="""
        var data = source.data
        var province = cb_obj.label
        var index = data['provinces'].indexOf(province)
        var results = cb_obj.value.split("|")
        var newlegend = results[0]
        var newColor = results[1]
        colors = data['colors']
        colors[index] = newColor
        legends = data['legends']
        legends[index] = newlegend
        if (province == '河北') {
            colors[index+1] = newColor
            legends[index+1] = newlegend
        }
        source.trigger('change')
    """)

dropdowns1 = []
dropdowns2 = []
dropdowns3 = []
def get_menu(row, time):
    menu = []
    variable = ['CloseContact', 'Confirmed', 'Critical', 'Cured', 'Dead']
    for item in variable:
        col = time+'_'+item
        menu.append(item+': '+str(data.loc[row,col]))
        # menu.append((item,str(data.loc[row,col])))
    return menu



for i in range(0,12):
    menu = get_menu(provinceArray[i],time)
    dropdown = Dropdown(label=provinceArray[i], menu=menu, callback = callback)
    dropdowns1.append(dropdown)
for i in range(12,24):
    menu = get_menu(provinceArray[i],time)
    dropdown = Dropdown(label=provinceArray[i], menu=menu, callback = callback)
    dropdowns2.append(dropdown)
for i in range(24,34):
    menu = get_menu(provinceArray[i],time)
    dropdown = Dropdown(label=provinceArray[i], menu=menu, callback = callback)
    dropdowns3.append(dropdown)

'''
data = dict(
        provinces=provinceArray,
        legends=legends
    )
tablesource = ColumnDataSource(data)
columns = [
        TableColumn(field="provinces", title="Province"),
        TableColumn(field="legends", title="Legend"),
    ]
data_table = DataTable(source=tablesource, columns=columns, width=400, height=400)
'''


output_file("chinamap.html", title = "China Map")
l = layout([
    [p,widgetbox(dropdowns1, width=100),widgetbox(dropdowns2, width=100),widgetbox(dropdowns3, width=100)]])
show(l)